In [2]:
! pip install -U bitsandbytes accelerate transformers peft trl --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, BitsAndBytesConfig, GenerationConfig
import bitsandbytes
import torch
from peft import LoraConfig, TaskType, get_peft_model, PeftModel
from tqdm import tqdm
from trl import SFTTrainer

## Import the data from Hugging face

In [5]:
dataset = load_dataset('samhog/psychology-10k', split = ['train'])[0]
# dataset.save_to_disk('./dataset')

# dataset = load_from_disk('./dataset')
dataset

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Psychology-10K.json:   0%|          | 0.00/5.21M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 9846
})

let's walk trough the data

In [6]:
def print_conv(me, model):
    for i, j in zip(me, model):
        print(
            f"""
Me : {i}

Param mitr : {j}\n\n
            """
        )

In [7]:
print_conv(dataset['input'][:3], dataset['output'][:3])


Me : I'm feeling really anxious lately and I don't know why.

Param mitr : It's common to feel anxious at times, and there can be many reasons for it. Have there been any recent changes or stressors in your life that may be contributing to your anxiety? Let's work together to identify any triggers and develop coping strategies to manage your anxiety.


            

Me : I think my partner may be cheating on me. What should I do?

Param mitr : It's understandable to feel worried and suspicious in this situation. Have you talked to your partner about your concerns? It's important to communicate openly and honestly with them. If you're still feeling uncertain, we can work on developing a plan to address the situation in a healthy and constructive way.


            

Me : I'm feeling really overwhelmed with work and school. I don't know how to manage my time and it's causing me a lot of stress.

Param mitr : It sounds like you're going through a difficult time. Let's work on creating a 

In [8]:
dataset.shape

(9846, 3)

In [9]:
Dataset = dataset.train_test_split(test_size = 0.2)
Dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 7876
    })
    test: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 1970
    })
})

## Import our model with its tokenizer

In [10]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

In [12]:
# Importing model from the Hugging Face hub

model_name = 'google/flan-t5-small'
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype = torch.bfloat16,
    quantization_config = bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Saving the model

# model.save_pretrained('/kaggle/working/Model')
# tokenizer.save_pretrained('/kaggle/working/Model')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [42]:
import gc
def clean():
    for i in range(15):
      gc.collect()
      torch.cuda.empty_cache()

In [43]:
# Importing model saved locally
# tokenizer = AutoTokenizer.from_pretrained('/kaggle/working/Model', device_map="auto")
# model = AutoModelForSeq2SeqLM.from_pretrained('/kaggle/working/Model',
#                                               device_map="auto",
#                                               torch_dtype = torch.bfloat16,
#                                               quantization_config = bnb_config
#                                              )

Let's try some stuff with tokenizer

In [44]:
tokenizer("hey", return_token_type_ids = False)

{'input_ids': [3, 13133, 1], 'attention_mask': [1, 1, 1]}

In [45]:
device = torch.device('cuda')
tokenizer.decode(
    model.generate(
        tokenizer(
            dataset['input'][0], return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        )['input_ids'].to(device)
    )[0], skip_special_tokens= True
)


"I'm feeling really anxious lately and I don't know why."

In [58]:
def inference(input_data, param_mitr = model):
    intruct = dataset['instruction'][0]
    inp = [intruct + "\n\n" + sent for sent in input_data]
    output = param_mitr.generate(
        tokenizer(
            inp, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        )['input_ids'].to(device)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)



In [59]:
inference(dataset['input'][:4])


Me : I'm feeling really anxious lately and I don't know why.

Param mitr : It's important to provide a helpful response to their concern. Let's work together to


            

Me : I think my partner may be cheating on me. What should I do?

Param mitr : It's important to provide a helpful response to the conflict. Consider seeking a therapist


            

Me : I'm feeling really overwhelmed with work and school. I don't know how to manage my time and it's causing me a lot of stress.

Param mitr : Psychologists can help you identify the source of your stress and develop coping strategies to manage your time


            

Me : I'm having trouble sleeping and I'm constantly tired. I think it might be because of my medication.

Param mitr : Psychotherapy can be helpful in addressing any underlying issues that may be contributing to your sleep difficulties


            


The model is showing good result on ICL (In Context Learning) with zero shot inference.

In [48]:
def tokenize_function(example):
    example['input_ids'] = tokenizer(example['input'], padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example['output'], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example


tokenized_datasets = Dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['input', 'output', 'instruction'])

Map:   0%|          | 0/1970 [00:00<?, ? examples/s]

## Fine tune our model


In [49]:
config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=16,
    target_modules = "all-linear",
    lora_alpha=32,
    lora_dropout=0.05
)

In [50]:
Model = get_peft_model(model, config)
Model.print_trainable_parameters()

trainable params: 2,555,904 || all params: 79,517,056 || trainable%: 3.2143


I have already saved the model and will not retrieve it from the hub

In [51]:
TrainArgs = TrainingArguments(
    output_dir = './Model',
    learning_rate = 1e-3,
    num_train_epochs = 2,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    weight_decay = 0.01,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps=1,
    load_best_model_at_end = True,
    report_to = ['tensorboard']
)

In [52]:
trainer = Trainer(
    model = Model,
    args = TrainArgs,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['test']
)

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [54]:
clean()
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.648400,1.628664
2,1.656200,1.615522


TrainOutput(global_step=3938, training_loss=1.7115088560182834, metrics={'train_runtime': 2002.4439, 'train_samples_per_second': 7.866, 'train_steps_per_second': 1.967, 'total_flos': 3051827792707584.0, 'train_loss': 1.7115088560182834, 'epoch': 2.0})

In [19]:
def peft_infrence(input_data, model = Model):
    output = model.generate(
        **tokenizer(
            input_data, return_token_type_ids = False, return_tensors = 'pt', padding=True, truncation=True
        ),
        generation_config = GenerationConfig(max_new_tokens=100, num_beams=1)
    )
    decoded = [tokenizer.decode(out, skip_special_tokens= True) for out in output]
    print_conv(input_data, decoded)

In [90]:
peft_infrence(dataset['input'][-4:])


Me : I feel like I'm stuck in a rut and don't know how to move forward.

Param mitr : It's important to identify the root cause of your stuck in a rut and develop coping strategies to manage it. Let's work together to identify the root cause of your stuck in a rut and develop a plan to move forward. We can also explore any underlying issues that may be contributing to your stuck in a rut.


            

Me : I'm having trouble connecting with my teenage daughter.

Param mitr : Relationship issues can be challenging, but it's important to communicate openly and honestly with your daughter. Let's work together to develop a plan to connect with your daughter and develop a plan to connect with her.


            

Me : I'm struggling with addiction and don't know where to turn.

Param mitr : Addiction is a complex issue, but it's important to seek help and support. Let's work together to identify the root cause of your addiction and develop a treatment plan that works for you. We can als

In [15]:
trainer.model.save_pretrained('./peft_for_param_mitr')